In [12]:
# Import

import os
from pathlib import Path
from shapely.geometry import Point
import numpy as np
import pandas as pd
import geopandas as gpd
import leafmap.foliumap as leafmap

In [13]:
# Pathing

current = Path.cwd() 
parent = current.resolve().parent / 'data'
os.chdir(parent)
os.getcwd()

'C:\\Users\\alons\\OneDrive\\Documents\\Code\\Python\\HSS350\\data'

In [14]:
MedInc22_AMI22_BG22_geo = gpd.read_file("MedInc22_AMI22_BG22_geo.geojson")
MedInc22_AMI22_BG22_geo['AMI22_3Family'].unique()

array(['N/A', 'Moderate', 'Low', 'Very Low', 'Extremely Low', 'Middle',
       'High and Above'], dtype=object)

In [11]:
MedInc22_AMI22_BG22_geo['AMI22_3Family_Ranked'] = ''

MedInc22_AMI22_BG22_geo.loc[MedInc22_AMI22_BG22_geo['AMI22_3Family'] == 'N/A', 'AMI22_3Family_Ranked'] = '1. N/A'
MedInc22_AMI22_BG22_geo.loc[MedInc22_AMI22_BG22_geo['AMI22_3Family'] == 'Extremely Low', 'AMI22_3Family_Ranked'] = '2. Extremely Low'
MedInc22_AMI22_BG22_geo.loc[MedInc22_AMI22_BG22_geo['AMI22_3Family'] == 'Very Low', 'AMI22_3Family_Ranked'] = '3. Very Low'
MedInc22_AMI22_BG22_geo.loc[MedInc22_AMI22_BG22_geo['AMI22_3Family'] == 'Low', 'AMI22_3Family_Ranked'] = '4. Low'
MedInc22_AMI22_BG22_geo.loc[MedInc22_AMI22_BG22_geo['AMI22_3Family'] == 'Moderate', 'AMI22_3Family_Ranked'] = '5. Moderate'
MedInc22_AMI22_BG22_geo.loc[MedInc22_AMI22_BG22_geo['AMI22_3Family'] == 'Middle', 'AMI22_3Family_Ranked'] = '6. Middle'
MedInc22_AMI22_BG22_geo.loc[MedInc22_AMI22_BG22_geo['AMI22_3Family'] == 'High and Above', 'AMI22_3Family_Ranked'] = '7. High and Above'
#MedInc22_AMI22_BG22_geo['Rank'] = MedInc22_AMI22_BG22_geo['Rank'].astype(int)
MedInc22_AMI22_BG22_geo.to_file(filename="MedInc22_AMI22_BG22_geo.geojson", driver="GeoJSON")

In [3]:
# # # Load

# MedInc22_CT20 = pd.read_csv("NYC_B19013_CT20.csv")
MedInc22_BG22 = pd.read_csv("NYC_B19013_BG22.csv")

# # # Spatial
# #Pluto = gpd.read_file("PLUTO_geo.geojson")
# NYC_CT20 = gpd.read_file("NYC_CT20.geojson")

NYC_BG22 = gpd.read_file("NYC_BG22.geojson")



In [4]:
# Clean

MedInc22_BG22 = MedInc22_BG22.drop(MedInc22_BG22.index[0])
MedInc22_BG22['GEO_ID'] = MedInc22_BG22['GEO_ID'].str.slice(start=9) 
MedInc22_BG22 = MedInc22_BG22[['GEO_ID', 'B19013_001E']]
MedInc22_BG22.rename(columns={'GEO_ID': 'GEOID'}, inplace=True)
MedInc22_BG22.to_csv("MedInc_BG22.csv")
MedInc22_BG22.to_csv("MedInc_BG22_control.csv")

In [5]:
# Conditional Columns

MedInc22_BG22['AMI22_3Family'] = ''


#MedInc22_BG22[(MedInc22_BG22['CompltYear'].between(2012, 2022))]

# Preliminary columns
MedInc22_BG22.loc[MedInc22_BG22['B19013_001E'] == '-', 'AMI22_3Family'] = 'N/A'
MedInc22_BG22.loc[MedInc22_BG22['B19013_001E'] == '2,500-', 'AMI22_3Family'] = 'Extremely Low'
MedInc22_BG22.loc[MedInc22_BG22['B19013_001E'] == '250,000+', 'AMI22_3Family'] = 'High and Above'

# Dummies

MedInc22_BG22['B19013_001E'] = MedInc22_BG22['B19013_001E'].replace('-', 0)
MedInc22_BG22['B19013_001E'] = MedInc22_BG22['B19013_001E'].replace('2,500-', 0)
MedInc22_BG22['B19013_001E'] = MedInc22_BG22['B19013_001E'].replace('250,000+', 0)
MedInc22_BG22['B19013_001E'] = MedInc22_BG22['B19013_001E'].astype(int)

# Ordinal AMI22 Columns 

# Extremely Low
MedInc22_BG22.loc[(MedInc22_BG22['B19013_001E'].between(1, 36030)), 'AMI22_3Family'] = 'Extremely Low'
MedInc22_BG22.loc[(MedInc22_BG22['B19013_001E'].between(36031, 60050)), 'AMI22_3Family'] = 'Very Low'
MedInc22_BG22.loc[(MedInc22_BG22['B19013_001E'].between(60051, 96080)), 'AMI22_3Family'] = 'Low'
MedInc22_BG22.loc[(MedInc22_BG22['B19013_001E'].between(96081, 144120)), 'AMI22_3Family'] = 'Moderate'
MedInc22_BG22.loc[(MedInc22_BG22['B19013_001E'].between(144121, 192160)), 'AMI22_3Family'] = 'Middle'
MedInc22_BG22.loc[(MedInc22_BG22['B19013_001E'].ge(192161)), 'AMI22_3Family'] = 'High and Above'

MedInc22_BG22_with_dummies = MedInc22_BG22

MedInc22_BG22.drop(columns=[MedInc22_BG22.columns[1]], inplace=True)

MedInc22_AMI22_BG22_geo = pd.merge(left=MedInc22_BG22,
                                   right=NYC_BG22,
                                   how='left',
                                   on='GEOID')

MedInc22_AMI22_BG22_geo = gpd.GeoDataFrame(MedInc22_AMI22_BG22_geo,
                                           geometry='geometry',
                                           crs=4326
                                           )
#MedInc22_AMI22_BG22_geo.to_file(filename='MedInc22_AMI22_BG22_geo.geojson', driver='GeoJSON')



In [6]:
# # Map
# m = leafmap.Map(#center=[40.7248387,-73.9775537]
#                 zoom=12, tiles="CartoDB.Positron")
# m.add_data(FC_421a_prep_ResUnit_CT20_geo,
#            column='UnitCount_by_CT20',
#            scheme='NaturalBreaks',
#            k=5,
#            info_mode='on_hover')
# m

In [ ]:
# MedInc22 Stuff

# # MedInc22_CT_geo
# MedInc22_CT20_geo = pd.merge(left=NYC_CT20,
#                              right=MedInc22_CT20,
#                              how='left',
#                              on='GEOID')
# MedInc22_CT20_geo = MedInc22_CT20_geo[['GEOID',
#                                        'B19013_001E',
#                                        'geometry']]
# MedInc22_CT20_geo['B19013_001E'] = MedInc22_CT20_geo['B19013_001E'].fillna(0)
# MedInc22_CT20_geo['B19013_001E'] = MedInc22_CT20_geo['B19013_001E'].astype(int)

# MedInc22_CT20_geo = gpd.GeoDataFrame(data=MedInc22_CT20_geo,
#                                      geometry='geometry',
#                                      crs=4326)

# # MedInc22_BG22_geo
# MedInc22_BG22_geo = pd.merge(left=NYC_BG22,
#                              right=MedInc22_BG22,
#                              how='left',
#                              on='GEOID')

# MedInc22_BG22_geo = MedInc22_BG22_geo[['GEOID',
#                                        'B19013_001E',
#                                        'geometry']]
# MedInc22_BG22_geo.to_csv("MedInc22_BG22_geo.csv", index=False)

# MedInc22_BG22_geo = MedInc22_BG22_geo[~MedInc22_BG22_geo['B19013_001E'].astype(str).str.contains('-', na=False)]
# MedInc22_BG22_geo['B19013_001E'].astype(int)

# MedInc22_BG22_geo = gpd.GeoDataFrame(data=MedInc22_BG22_geo,
#                                      geometry='geometry',
#                                      crs=4326)


# MedInc22_CT20_geo.dropna(inplace=True)
# FC_421a_prep_ResUnit_by_MedInc22_CT20 = pd.merge(left=FC_421a_prep_ResUnit_CT20,
#                                                  right=FC_421a_prep_ResUnit_CT20,
#                                                  how='left',
#                                                  on='GEOID')
